<a href="https://colab.research.google.com/github/PoliNemkova/conflict_prediction_2025/blob/main/Baseline_XGBoost_Myanmar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
country = 'Myanmar'
country_code_2 = 'MM'
country_code_3 = 'MMR'

In [2]:
import pandas as pd
from google.colab import drive  # to mount Drive to Colab notebook
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#hoa = pd.read_csv('/content/gdrive/MyDrive/CONFLICT PREDICTION/Data/HOA/HornsofAfrica_since2020_events+text+encoding.csv')
data = pd.read_csv('/content/gdrive/MyDrive/CONFLICT PREDICTION/Data/Myanmar/Myanmar_year2020-_2025_gdelt_articles_texts.csv')

In [4]:
gdelt_filtered = data[
    data['actor1countrycode'].isin([country, country_code_2, country_code_3]) |
    data['actor2countrycode'].isin([country, country_code_2, country_code_3]) |
    data['actiongeocountrycode'].isin([country, country_code_2, country_code_3])
]
len(gdelt_filtered)

2214

In [ ]:
gdelt_filtered.head(2)

,Unnamed: 0,index,globaleventid,sqldate,monthyear,year,fractiondate,actor1code,actor1name,actor1countrycode,...,actiongeoadm1code,actiongeoadm2code,actiongeolat,actiongeolong,actiongeofeatureid,dateadded,sourceurl,geometry,date,text
0,0,3,896925286,20200105,202001,2020,2020.0137,MMR,MYANMAR,MMR,...,CA,NaN,60.0,-96.0,CA,20200105234500,https://www.bangkokpost.com/opinion/opinion/18...,POINT (-96 60),20200105,Why is Suu Kyi defending the military?\n\nOn D...
1,1,4,896925287,20200105,202001,2020,2020.0137,MMRCVL,MYANMAR,MMR,...,BG,NaN,24.0,90.0,BG,20200105234500,https://www.bangkokpost.com/opinion/opinion/18...,POINT (90 24),20200105,Why is Suu Kyi defending the military?\n\nOn D...


In [5]:
gdelt_filtered.drop_duplicates(subset='text', inplace=True)
len(gdelt_filtered)

478

In [28]:
gdelt_filtered.reset_index(drop=True, inplace=True)

# Filtering and Summarization

In [6]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.82.1
    Uninstalling openai-1.82.1:
      Successfully uninstalled openai-1.82.1


In [7]:
import openai
import json
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
keys = pd.read_csv('/content/gdrive/MyDrive/Computing/api_keys.csv')
OPENAI_API_KEY = keys['key'][keys['api']=='openai'][0]
openai.api_key = OPENAI_API_KEY

In [15]:
# filtering with GPT $15 for 478 texts

import openai

def summarize_with_gpt(df, country, max_rows=50):
    """
    Filter GDELT data using GPT.

    Args:
        df (pd.DataFrame): DataFrame containing at least 'sqldate', 'goldsteinscale', 'tone', and 'title' or 'event' column.
        max_rows (int): Limit number of rows to avoid overly long prompts.

    Returns:
        str: GPT-generated label.
    """
    if df.empty:
        return "DF is empty"


    texts = df.tolist()
    labels = []

    for i in range(len(texts)):
      print('============> Started filtering...')
      print(f'============> Total length ot texts to process: {len(texts)}')

      prompt = f'''You are a geopolitical analyst specializing in conflict monitoring.
      Read the following news article or event description and determine whether it is directly related to conflict ,
      political violence, civil unrest, or rising tensions in {country}.

      Label the article as follows:
      1 if it clearly involves or signals conflict, political violence, protests, repression, military activity, or diplomatic escalation.
      0 if it does not directly relate to any of the above (e.g., business, sports, general politics, culture, or commentary with no conflict signals).

      Return only the number 1 or 0 as your answer. No explanation.

      Article:
      {text[i]}'''

      try:
          response = openai.ChatCompletion.create(
              model="gpt-4",
              messages=[{"role": "user", "content": prompt}],
              temperature=0.5,
              max_tokens=300
          )
          labels.append(response.choices[0].message['content'].strip())
      except Exception as e:
          labels.append(f"[Error summarizing with GPT: {str(e)}")

      if i % 100 == 0:
          print(f"============> Processed {i} texts")

    return labels

In [16]:
results = summarize_with_gpt(gdelt_filtered['text'], country) #took ~5 mins for ~500 texts

In [17]:
results = pd.DataFrame(results, columns=['label'])

results.value_counts()

,count
label,
1,345
0,132
[Error summarizing with GPT: The server is overloaded or not ready yet.,1


In [29]:
len(results)

478

In [30]:
len(gdelt_filtered)

478

In [31]:
gdelt_filtered['label'] = results

In [25]:
results

,label
0,1
1,[Error summarizing with GPT: The server is ove...
2,1
3,1
4,1
...,...
473,1
474,1
475,0
476,1


In [35]:
gdelt_filtered_llmed = gdelt_filtered[gdelt_filtered['label'] == '1']

In [36]:
len(gdelt_filtered_llmed)

345

In [ ]:
# summarization with GPT (too expensive)
"""
import openai

def summarize_with_gpt(df, country, max_rows=50):
    """"""
    Summarize a week of GDELT data using GPT.

    Args:
        df (pd.DataFrame): DataFrame containing at least 'sqldate', 'goldsteinscale', 'tone', and 'title' or 'event' column.
        max_rows (int): Limit number of rows to avoid overly long prompts.

    Returns:
        str: GPT-generated summary string.
    """"""
    if df.empty:
        return "DF is empty for this week"


    rows_text = df

    joined_text = "\n".join(rows_text)

    prompt = f'''You are a conflict analyst. The following is a list of news event records from the past week.
    Your task is to write one concise analytical summary that includes only the articles relevant to
    current or emerging conflict dynamics in {country}. Focus on political violence, military actions, protests, repression,
    escalation risks, and tensions between actors.\n\n
    Do not include articles unrelated to conflict (such as business, culture, sports, etc.).
    Prioritize information about:\n
    - Armed clashes, attacks, or military movements\n
    - Political unrest, civil protests, or riots\n
    - Repressive state actions (e.g., censorship, arrests)\n
    - Diplomatic tensions or warnings\n
    - Any patterns suggesting rising instability\n\n
    Summarize as a single paragraph or two, in clear, analytical language. If nothing identifies, return "No events". \n\n
    Here are the events:\n---\n
    {joined_text}\n
    ---\n\n
    Weekly Conflict Summary: '''

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=300
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"[Error summarizing with GPT: {str(e)}]"
"""

In [37]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `LLM annotations study` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re

In [ ]:
# another version of summarization function but with LLAMA

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load the tokenizer and model once (outside the function ideally for efficiency)
model_name = "meta-llama/Llama-2-13b-chat-hf"  # or use another chat-tuned LLaMA variant
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
summarizer = pipeline("text-generation", model=model, tokenizer=tokenizer)

def summarize_with_llama(df, country, max_rows=50):
    """
    Summarize a week of GDELT data using LLaMA (via Hugging Face Transformers).

    Args:
        df (pd.DataFrame): DataFrame with 'text' column or similar.
        country (str): Country name to focus the summary on.
        max_rows (int): Number of rows to include.

    Returns:
        str: Summary string.
    """
    if df.empty:
        return "DF is empty for this week"

    # Limit and join rows
    rows_text = df.head(max_rows)['text'].dropna().astype(str).tolist()
    joined_text = "\n".join(rows_text)

    # Prompt template
    prompt = f"""<s>[INST] You are a conflict analyst. The following is a list of news event records from the past week.
Your task is to write one concise analytical summary. Summarize as a single paragraph or two, in clear, analytical language.

Here are the texts:
---
{joined_text}
---

Weekly Conflict Summary: [/INST]"""

    try:
        output = summarizer(prompt, max_new_tokens=300, temperature=0.5, do_sample=True)[0]['generated_text']
        # Post-process to remove any extra tokens if needed
        if "Weekly Conflict Summary:" in output:
            return output.split("Weekly Conflict Summary:")[-1].strip()
        return output.strip()
    except Exception as e:
        return f"[Error summarizing with LLaMA: {str(e)}]"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

In [ ]:
from datetime import timedelta
import pandas as pd

# Make sure date column is datetime
gdelt_filtered['sqldate'] = pd.to_datetime(gdelt_filtered['sqldate'], format="%Y%m%d", errors="coerce")
#gdelt_filtered.loc[:, 'sqldate'] = pd.to_datetime(gdelt_filtered['sqldate'])



# Set custom start and end dates (you will provide these)
start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2025-01-01")  # inclusive

# Ensure start_date is the first of the month and end_date is the last of the month
start_date = start_date.replace(day=1)
end_date = end_date + pd.offsets.MonthEnd(0)

# Initialize outputs
monthly_summaries = []
monthly_summary_start_dates = []
monthly_summary_end_dates = []

current_start = start_date

while current_start <= end_date:
    current_end = current_start + pd.offsets.MonthEnd(0)

    # Filter data for the current month
    monthly_data = gdelt_filtered[
        (gdelt_filtered['sqldate'] >= current_start) & (gdelt_filtered['sqldate'] <= current_end)
    ]
    #print(f"Checking month: {current_start.date()} to {current_end.date()}")
    #print(f"Events this month: {len(monthly_data)}")
    #break

    if monthly_data.empty:
        current_start += pd.DateOffset(months=1)
        continue

    # Summarize by week inside the month
    monthly_summary = ''
    week_start = current_start
    week_end = week_start + timedelta(days=6)

    while week_start <= current_end:
        week_data = monthly_data[
            (monthly_data['sqldate'] >= week_start) & (monthly_data['sqldate'] <= min(week_end, current_end))
        ]

        #print(f"Checking week: {week_start.date()} to {min(week_end, current_end).date()}")
        #print(f"Events this week: {len(week_data)}")
        #print(week_data)
        #break

        # Pass DataFrame directly (summarize_with_gpt expects df)
        #weekly_summary = summarize_with_gpt(week_data['text'], country)
        weekly_summary = summarize_with_llama(week_data, country)

        monthly_summary += '| ' + weekly_summary

        week_start = week_end + timedelta(days=1)
        week_end = week_start + timedelta(days=6)

    # Store results
    monthly_summaries.append(monthly_summary)
    monthly_summary_start_dates.append(current_start)
    monthly_summary_end_dates.append(current_end)

    current_start += pd.DateOffset(months=1)

# Combine into DataFrame
monthly_summary_df = pd.DataFrame({
    'start_date': monthly_summary_start_dates,
    'end_date': monthly_summary_end_dates,
    'summary': monthly_summaries
})

   Unnamed: 0  index  globaleventid    sqldate  monthyear  year  fractiondate  \
0           0      3      896925286 2020-01-05     202001  2020     2020.0137   

  actor1code actor1name actor1countrycode  ... actiongeoadm1code  \
0        MMR    MYANMAR               MMR  ...                CA   

  actiongeoadm2code actiongeolat actiongeolong actiongeofeatureid  \
0               NaN         60.0         -96.0                 CA   

        dateadded                                          sourceurl  \
0  20200105234500  https://www.bangkokpost.com/opinion/opinion/18...   

         geometry      date                                               text  
0  POINT (-96 60)  20200105  Why is Suu Kyi defending the military?\n\nOn D...  

[1 rows x 67 columns]
   Unnamed: 0  index  globaleventid    sqldate  monthyear  year  fractiondate  \
9           9     19      898572500 2020-01-12     202001  2020     2020.0329   

  actor1code actor1name actor1countrycode  ... actiongeoadm1code  \


In [ ]:
monthly_summary_df

# encoding

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('intfloat/e5-base-v2')
summary = "passage: " + monthly_summary
embedding = model.encode(summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

# XGBoost


In [ ]:
# import labels

acled = pd.read_csv('/content/gdrive/MyDrive/CONFLICT PREDICTION/Data/Myanmar/acled_2020_2025.csv')

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

def train_xgboost_high_dim(df, label_col='label', cv_folds=5, random_state=42):
    # Separate features and label
    X = df.drop(columns=[label_col])
    y = df[label_col]

    # Create XGBoost classifier with regularization
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        max_depth=2,           # very shallow trees
        learning_rate=0.1,
        n_estimators=50,
        reg_alpha=1,           # L1 regularization
        reg_lambda=1,          # L2 regularization
        subsample=0.8,         # for regularization
        colsample_bytree=0.8,  # use 80% features per tree
        random_state=random_state
    )

    # Cross-validation
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

    print(f"Cross-validated accuracy (mean over {cv_folds} folds): {scores.mean():.4f}")
    print("All fold accuracies:", scores)

    # Train final model on full data
    model.fit(X, y)

    return model


In [ ]:
trained_model = train_xgboost_high_dim(df, label_col='label')